# **KDDM 2 SARTORIUS CELL INSTANCE SEGMENTATION MMCLASSIFICATION TRAIN NOTEBOOK**

# **INSTALL All DEPENDENCIES**
* **Install PyTorch version required by MMDetection**
* **Install MMClassification and required dependencies**

In [ ]:
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torch-1.7.0+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchvision-0.8.1+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/pytorch-170-cuda-toolkit-110221/torchaudio-0.7.0-cp37-cp37m-linux_x86_64.whl' --no-deps

In [ ]:
!pip install '/kaggle/input/mmdetectionv2140/addict-2.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/yapf-0.31.0-py2.py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/terminal-0.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/mmdetectionv2140/terminaltables-3.1.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/kddm2/mmcls/mmcv_full-1.3.18-cp37-cp37m-manylinux1_x86_64.whl' --no-deps

!cp -r /kaggle/input/mmdetectionv2140/pycocotools-2.0.2/* /kaggle/working/
!cp -r /kaggle/input/mmdetectionv2140/mmpycocotools-12.0.3/* /kaggle/working/

!pip install '/kaggle/working/mmpycocotools-12.0.3/' --no-deps
!pip install '/kaggle/working/pycocotools-2.0.2/' --no-deps

!rm -rf /kaggle/working/pycocotools-2.0.2/
!rm -rf /kaggle/working/mmpycocotools-12.0.3/

!rm -rf mmdetection

!cp -r /kaggle/input/kddm2/mmcls/mmclassification-0.18.0/mmclassification-0.18.0 /kaggle/working/mmclassification/
%cd /kaggle/working/mmclassification
!pip install -e .

* **Check Installation**

In [ ]:
import torch
print(torch.__version__, torch.cuda.is_available(),  torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'NO CUDA DEVICE')
!nvcc --version

# Check mmcv installation
import mmcv
print(mmcv.__version__)

# Check MMClassification installation
import mmcls
print(mmcls.__version__)


from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print("Cuda:", get_compiling_cuda_version(), "Compiler:", get_compiler_version())

# **TRAINING**
* **Define model and settings for training**
* **Execute training**

* **Set MMDetection model config**

In [ ]:
#choose a configuration

#base_cfg = "/kaggle/working/mmclassification/configs/resnet/resnet50_8xb32_in1k.py"
base_cfg = "/kaggle/working/mmclassification/configs/resnext/resnext50-32x4d_8xb32_in1k.py"
#load config
cfg = mmcv.Config.fromfile(base_cfg)

###############################################################################################################

#update some parameters in the config
cfg["model"]["head"]["num_classes"] = 3
#cfg["model"]["backbone"]["init_cfg"] = dict(type='Pretrained', checkpoint="https://download.openmmlab.com/mmclassification/v0/resnet/resnet50_8xb32_in1k_20210831-ea4938fc.pth", prefix='backbone')
cfg["model"]["backbone"]["init_cfg"] = dict(type='Pretrained', checkpoint="https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth", prefix='backbone')
cfg["model"]["head"]["topk"] = 1

cfg["data"]["train"]["data_prefix"] = "/kaggle/input/sartorius-cell-instance-segmentation/train/"
cfg["data"]["val"]["data_prefix"] = "/kaggle/input/sartorius-cell-instance-segmentation/train/"
cfg["data"]["test"]["data_prefix"] = "/kaggle/input/sartorius-cell-instance-segmentation/train/"
cfg["data"]["train"]["ann_file"] = "/kaggle/input/kddm2/mmcls/cls_train.txt"
cfg["data"]["val"]["ann_file"] = "/kaggle/input/kddm2/mmcls/cls_val.txt"
cfg["data"]["test"]["ann_file"] = "/kaggle/input/kddm2/mmcls/cls_val.txt"

cfg["data"]["train"]["pipeline"][1] = dict(type='RandomResizedCrop', size=300, backend='pillow')
cfg["data"]["val"]["pipeline"][1] = dict(type='Resize', size=(520, -1), backend='pillow')
cfg["data"]["val"]["pipeline"][2] = dict(type='CenterCrop', crop_size=300)
cfg["data"]["test"]["pipeline"][1] = dict(type='Resize', size=(520, -1), backend='pillow')
cfg["data"]["test"]["pipeline"][2] = dict(type='CenterCrop', crop_size=300)

cfg["lr_config"] = dict(policy='step', step=[65, 90])
cfg["runner"] = dict(type='EpochBasedRunner', max_epochs=140)
cfg["optimizer"] = dict(type='SGD', lr=0.08, momentum=0.9, weight_decay=0.0001)

cfg["checkpoint_config"] = dict(interval=5)
cfg["evaluation"] = dict(interval=5, metric='accuracy')

###############################################################################################################

#store updated config file
cfg_file = "/kaggle/working/mmcls_sartorius.py"
with open(cfg_file, "w") as f:
    f.write(cfg.pretty_text)
print(cfg.pretty_text)

* **Train model**

In [ ]:
!python tools/train.py $cfg_file

# **TEST INFERENCE AND PERSIST TRAINED MODEL WEIGHTS**

In [ ]:
from mmcls.apis import init_model, inference_model
import os

!ls /kaggle/working/mmclassification/work_dirs/mmcls_sartorius -lh
best_epoch = "/kaggle/working/mmclassification/work_dirs/mmcls_sartorius/epoch_10.pth"
model = init_model(cfg_file, best_epoch)

test_dir = "/kaggle/input/sartorius-cell-instance-segmentation/test"
files = os.listdir(test_dir)

for f in files:
    img = os.path.join(test_dir, f)
    res = inference_model(model, img)
    print("{:s}: class {:d} [conf: {:.3f}]".format(f, res["pred_label"], res["pred_score"]))
    

* **Prepare best epoch for download**

In [ ]:
!cp $best_epoch /kaggle/working/mmcls_resnext50_0.9918_sartorius.pth
cfg_name = cfg.file.split("/")[-1]
!cp $cfg_file /kaggle/working/$cfg_name
!ls /kaggle/working